### 关于爬虫库使用
#### 1. get函数使用：

In [7]:
# encoding='utf-8'
import requests

url='http://www.spiderbuf.cn/s01/'

# 通过get函数进行网页获取
dataSource=requests.get(url)
print(dataSource)  # 但是这里输出的值并不是解析完全的字符串

<Response [200]>


In [8]:
dataText=dataSource.text # 以字符串形式解析
print(dataText)

<!DOCTYPE html>
<html lang="zh-CN">

<head>
  
<meta charset="utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<link href="http://apps.bdimg.com/libs/bootstrap/3.2.0/css/bootstrap.min.css" rel="stylesheet" />
<link href="http://apps.bdimg.com/libs/bootstrap/3.2.0/css/bootstrap-theme.min.css" rel="stylesheet" />
<link href="/static/css/theme.min.css" rel="stylesheet" />
<meta name="keywords" content="python,python 爬虫,爬虫 python,爬虫,反爬,robots,爬虫 练习,爬虫 示例,爬虫 案例" />
<meta name="description" content="加入Python爬虫攻防技术社区，通过游戏化、亲身体验，将你的网络爬虫技能提升到一个新的水平。而这还仅仅是个开始。" />
<meta name="robots" content="all" />
<meta name="author" content="spiderbuf" />
<meta name="copyright" content="spiderbuf" />
<script>
    var _hmt = _hmt || [];
    (function () {
        var hm = document.createElement("script");
        hm.src = "https://hm.baidu.com/hm.js?ab0153f5f064a819a91a3699f63f86e3";
        var s = document.getElementsByT

#### 2. 解析html数据（`lxml`内的解析）：

In [9]:
from bs4 import BeautifulSoup

# 以lxml方式进行解析
soup=BeautifulSoup(dataText,'lxml')
print(soup)

<!DOCTYPE html>
<html lang="zh-CN">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://apps.bdimg.com/libs/bootstrap/3.2.0/css/bootstrap.min.css" rel="stylesheet"/>
<link href="http://apps.bdimg.com/libs/bootstrap/3.2.0/css/bootstrap-theme.min.css" rel="stylesheet"/>
<link href="/static/css/theme.min.css" rel="stylesheet"/>
<meta content="python,python 爬虫,爬虫 python,爬虫,反爬,robots,爬虫 练习,爬虫 示例,爬虫 案例" name="keywords"/>
<meta content="加入Python爬虫攻防技术社区，通过游戏化、亲身体验，将你的网络爬虫技能提升到一个新的水平。而这还仅仅是个开始。" name="description"/>
<meta content="all" name="robots"/>
<meta content="spiderbuf" name="author"/>
<meta content="spiderbuf" name="copyright"/>
<script>
    var _hmt = _hmt || [];
    (function () {
        var hm = document.createElement("script");
        hm.src = "https://hm.baidu.com/hm.js?ab0153f5f064a819a91a3699f63f86e3";
        var s = document.getElementsByTagName("script"

+ 筛选我们需要的数据`<tr>`内的文本，从这之中再筛出`<td>`内的文本<br>
    这里我使用`csv`的格式输出文本

In [21]:
s=''
for trs in soup.select('tr'):
    # 首先进行逐行解析
    tds=trs.select('td')
#     print(tds)
    tr_text=[i.get_text() for i in tds]
#     print(tr_text)
    s+=','.join(tr_text)+'\n'
print(s)


1,172.16.80.178,CD-82-76-71-65-75,堡垒机,服务器,Windows10,80,22,443,在线
2,172.25.144.37,E6-84-22-55-44-BE,摄像头,摄像头,HUAWEI,80,22,443,离线
3,192.168.33.144,37-01-AE-BE-5E-C0,文件服务器,服务器,Linux,80,22,443,在线
4,10.229.10.217,84-A9-97-A8-B2-99,交换机,交换机,HUAWEI,,离线
5,10.159.189.52,8C-94-9D-85-6C-C1,堡垒机,服务器,Windows10,80,22,443,在线
6,10.89.223.129,F2-10-E3-CA-DF-DC,数据库服务器,服务器,Windows10,80,22,443,在线
7,192.168.224.239,46-9A-AF-F4-DC-33,数据库服务器,服务器,Windows10,80,22,443,在线
8,10.9.161.97,B5-66-A6-0C-C6-57,堡垒机,服务器,Linux,80,22,443,在线
9,192.168.100.105,5C-3F-8E-6E-D9-C5,OA服务器,服务器,Linux,80,22,443,离线
10,192.168.250.163,EC-C6-79-88-4C-BA,测试服务器,服务器,Linux,80,22,443,离线



+ 封装获取部分的代码：

In [68]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd


# 截取头部
def get_head(data):
    row=data.find('tr')
    tds=row.select('th')
    head=[i.get_text() for i in tds]
#     print(head)
    return head

# 转换部分
def transfer_tab(data):
    s=[]
    for row in data.select('tr'):
        tds=row.select('td')
        s.append([i.get_text() for i in tds])
    return [get_head(data)]+s[1:]  # 因为缺少表格头

# 转换到csv文件
def save_file(path,data):
    with open(path,'w',encoding='utf-8') as fp:
        for i in data:
            fp.write(','.join(i)+'\n')
        fp.close()


# 获取和输出部分
def get_table_csv(path,url):
    data=requests.get(url).text
    data=bs(data,'lxml')
    li=transfer_tab(data)
#     print(s)
    save_file(path,li)
#     print('输出完成')
    
    
if __name__=='__main__':
    url='http://www.spiderbuf.cn/s01/'
    get_table_csv('01.csv',url)
    # 显示csv文本
    tab=pd.read_csv(r'01.csv',engine='python',encoding='utf-8')
    tab.reset_index(drop=True,inplace=True)
    display(tab)

['序号', 'IP地址', 'MAC地址', '设备名称', '设备类型', '操作系统', '开放端口', '在线状态']


,序号,IP地址,MAC地址,设备名称,设备类型,操作系统,开放端口,在线状态
0,CD-82-76-71-65-75,堡垒机,服务器,Windows10,80.0,22,443.0,在线
1,E6-84-22-55-44-BE,摄像头,摄像头,HUAWEI,80.0,22,443.0,离线
2,37-01-AE-BE-5E-C0,文件服务器,服务器,Linux,80.0,22,443.0,在线
3,84-A9-97-A8-B2-99,交换机,交换机,HUAWEI,NaN,离线,NaN,None
4,8C-94-9D-85-6C-C1,堡垒机,服务器,Windows10,80.0,22,443.0,在线
5,F2-10-E3-CA-DF-DC,数据库服务器,服务器,Windows10,80.0,22,443.0,在线
6,46-9A-AF-F4-DC-33,数据库服务器,服务器,Windows10,80.0,22,443.0,在线
7,B5-66-A6-0C-C6-57,堡垒机,服务器,Linux,80.0,22,443.0,在线
8,5C-3F-8E-6E-D9-C5,OA服务器,服务器,Linux,80.0,22,443.0,离线
9,EC-C6-79-88-4C-BA,测试服务器,服务器,Linux,80.0,22,443.0,离线
